# 키워드 입력 후 크롤링

In [16]:
from bs4 import BeautifulSoup
import requests
import re
import sys
import pprint
import csv
import json
import pandas as pd

result=[]

headers = {
    'authority': 'apis.naver.com',
    'accept': '*/*',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    # Requests sorts cookies= alphabetically
    # 'cookie': 'ab.storage.userId.7af503ae-0c84-478f-98b0-ecfff5d67750=%7B%22g%22%3A%22browser-1625985144309-6%22%2C%22c%22%3A1626101500089%2C%22l%22%3A1626101500089%7D; ab.storage.deviceId.7af503ae-0c84-478f-98b0-ecfff5d67750=%7B%22g%22%3A%224cbe130c-6edd-d4aa-a78d-290b003c3592%22%2C%22c%22%3A1626101500094%2C%22l%22%3A1626101500094%7D; ASID=7992e0220000017aaa36664e0000004e; _ga=GA1.2.612969395.1626832328; ab.storage.sessionId.7af503ae-0c84-478f-98b0-ecfff5d67750=%7B%22g%22%3A%2228148006-e01d-7623-b7d1-b4fff0f59b4e%22%2C%22e%22%3A1627919390179%2C%22c%22%3A1627908091281%2C%22l%22%3A1627917590179%7D; MM_NEW=1; NFS=2; NNB=RDIIILNX6JCWE; nx_ssl=2; nid_inf=1665554565; NID_AUT=tP3V5ox533EjyAgkJ1JaqWEnPOhXs2hr3teD39pK972fuXqDWQZXoIOMzICJpa1A; NID_JKL=d393brIzilbjw+7TVvG0OW6Eo22+WIhQAfihItUdgbY=; _naver_usersession_=SPdJTrlTMrn8Udkyn58eo6HL; NID_SES=AAABwJaKJ5FjUAETXL8SAX2HKMUSTt3l8pPu49OSzbGzgKEEMN/ckpP4DbQVHQwTV1hVPWtbpP7Nomg0CbD8TtCpyOYbeq8+OpHb5eWbDsXXCeLHO4epgthLtbQHiBE8spXqEtx/h0D6MzxsIlN4pa8gz51jV+oWzQQNnpQCeaKKLaxcpMfhGXnZv4BK1Rg+TAgUFE9RtExcKjteTL2hB9tKT41C7antdQdhLfVXWUbsJ/q5b62iDZnnZUAANXHnWp/9RI2YyKSn70SVu4Bag+fxA/23OqjCHSbK5RMiNOQKV+Bs7uugaAsMKkH6lGBBIbNDkTXGZ4n1+KbqFwe1kV9oCaPJ+siwXESEqvY0jaLVNAqUATQZjnIMFIYwARw41FTuduxW1IOF7MdP7R3EqOvnqNir2lXW1UfRlHlOtMC4w/tXk8xqJR/HVlZrnltKkMZB5zfyDNvnt02jbOKJcORjmOeVvL+xoCdSXwZclfJzRkC31l43+9jSu4X8RPUfuJILRMHf2e1A0NU7Mwds7h+S//5AD0yUJlPtFFzLvriuD1SMTRXiSwN4pNWBi6UIsPzScRpyLMc8hUE8Bi8jJtGk4e0=; NDARK=N; page_uid=hrKUflprvN8ssNc4Muwssssss3R-382317; BMR=',
    'referer': 'https://n.news.naver.com/article/028/0002595736',
    'sec-ch-ua': '"Whale";v="3", " Not;A Brand";v="99", "Chromium";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'script',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.108 Whale/3.15.136.18 Safari/537.36',
}

def news_url(query):
    page =1 
    url_list=[]
    while page <= 1001 :

        url = "https://m.search.naver.com/search.naver?where=m_news&sm=mtb_pge&query="+query+"&sort=0&photo=0&field=0&pd=3&ds=2022.12.29&de=2022.12.29&cluster_rank=37&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from20221229to20221229&start="+str(page)
        response = requests.get(url,headers=headers)
        html = response.text

        #뷰티풀소프의 인자값 지정
        soup = BeautifulSoup(html, "lxml")
        # print(soup.select_one('a.news_tit > div'))
        #<a>태그에서 제목과 링크주소 추출
        atags = soup.select('.news_tit')
        # title = soup.select_one('a.news_tit > div')
        # print(title)
        for i in atags:
            if "https://n.news.naver.com/" in i['href'] :
                url_list.append(i)   #링크주소
                # print(i)
        if len(atags) < 15 : #한페이지에 15개나오는데, 그보다 작으면 없다고 판단
            break
        page = page + 15
    return url_list
    # 위에서 크롤링한 네이버 뉴스 url을 입력합니다.
def comment(url_list):
    result = pd.DataFrame()
    total_comment = [] #모든 댓글을 담는 리스트
    for url_ex in url_list:
        url = url_ex['href']
        title = url_ex.select_one(' div')
        title = re.sub('<.+?>', '', str(title), 0, re.I|re.S)
        # print(title)
        url = url.split('?')[0]
        oid_1 = url.split('/')[-1]
        oid_2 = url.split('/')[-2]
        i = 1
        # df = pd.DataFrame(columns=['댓글'])
        while True:
            params = {
                'ticket': 'news',
                'templateId': 'default_society',
                'pool': 'cbox5',
                'lang': 'ko',
                'country': 'KR',
                'objectId': f'news{oid_2},{oid_1}',
                'pageSize': '100',
                'indexSize': '10',
                'page': str(i),
                'currentPage': '0',
                'moreParam.direction': 'next',
                'moreParam.prev': '10000o90000op06guicil48ars',
                'moreParam.next': '1000050000305guog893h1re',
                'followSize': '100',
                'includeAllStatus': 'true',
            }

            response = requests.get('https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json', params=params, headers=headers)
            response.encoding = "UTF-8-sig"
            res = response.text.replace("_callback(","")[:-2]
            temp=json.loads(res) 
            # print(temp)
            date = temp['date']
            # print(date)
            try : 
                comment = list(pd.DataFrame(temp['result']['commentList'])['contents'])
                # print(comment)
                for j in range(len(comment)):
                    values = pd.DataFrame.from_dict({"title":title,"comment":comment[j],"date":date},orient='index')
                    result =  pd.concat([result,values]) 
                if len(comment)<97: 
                    break 
                else: 
                    i+=1
            except :
                break    

               

    # print(total_comment)
    return result

In [15]:
# # 실행함수
def main():
    query = input("검색어 입력 : ")  
    # query='미국'
    url_list = news_url(query)  
    allCommetns = comment(url_list)

# query='미국'
query = input("검색어 입력 : ")  
url_list = news_url(query)

In [17]:
allCommetns = comment(url_list)

In [18]:
len(allCommetns)

57423

In [20]:
df = allCommetns.reset_index()
title = df[df['index'] == 'title'].reset_index().drop(['index','level_0'],axis=1).rename(columns={0:'title'})
comment = df[df['index'] == 'comment'].reset_index().drop(['index','level_0'],axis=1).rename(columns={0:'comment'})
date = df[df['index'] == 'date'].reset_index().drop(['index','level_0'],axis=1).rename(columns={0:'date'})
allCommetns = pd.concat([title,comment,date],axis=1)
len(allCommetns)

19141

In [28]:
allCommetns

,title,comment,date
0,"제2경인고속도로 방음터널 화재 완진…""5명 사망""",고인의 명복을 빕니다... 소방대원분들 수고많으십니다,2023-01-11T00:20:05+0000
1,"제2경인고속도로 방음터널 화재 완진…""5명 사망""",,2023-01-11T00:20:05+0000
2,"제2경인고속도로 방음터널 화재 완진…""5명 사망""",민주당아! 국정조사 준비해야지\n유가족 선동해서 또 정권탈취 시도해야지,2023-01-11T00:20:05+0000
3,"제2경인고속도로 방음터널 화재 완진…""5명 사망""",하필 터널에서 사고가 나서 피해가 크군요. 고인의 명복을 빕니다.\n어떻해ㅜㅜ,2023-01-11T00:20:05+0000
4,"제2경인고속도로 방음터널 화재 완진…""5명 사망""",매번 현장탓 하니 누가 현장에 뛰어들려고 할까?... 각자도생!!,2023-01-11T00:20:05+0000
...,...,...,...
19136,"[7NEWS] 中 비밀경찰서로 지목된 식당, “정상 영업장” 해명",꺼져라,2023-01-11T00:21:43+0000
19137,"[7NEWS] 中 비밀경찰서로 지목된 식당, “정상 영업장” 해명",중긕 공산당원 시진핑 식당 다들켰다 귀신의 집 무사 느가가냐 짜장...,2023-01-11T00:21:43+0000
19138,"[7NEWS] 中 비밀경찰서로 지목된 식당, “정상 영업장” 해명",진실이 밝혀진다? 이재명이 하는 소리가 뉘앙스가 똑 같구나.,2023-01-11T00:21:43+0000
19139,"[7NEWS] 中 비밀경찰서로 지목된 식당, “정상 영업장” 해명",,2023-01-11T00:21:43+0000


In [29]:
df_all = pd.DataFrame()

In [30]:
df_all = pd.concat([df_all,allCommetns])

In [31]:
df_all = df_all.drop_duplicates()

In [32]:
len(df_all)

14928

In [34]:
df_all.to_csv('comment_all_K.csv',encoding='utf-8-sig',index=False)